In [2]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import pandas as pd
import numpy as np

# Imports
import os
from google.cloud import bigquery

In [3]:
# Environment variables
gcp_project_id = os.environ['GCP_PROJECT']
gcp_service_account_key = os.environ['GCP_SERVICE_ACCOUNT_KEY']
bq_source_dataset = os.environ['BQ_SOURCE_DATASET']
bq_cleaned_dataset = os.environ['BQ_CLEANED_DATASET']

In [4]:
# Import data from BQ and set data to X
# Initialize a BigQuery client using the service account JSON file
bq_client = bigquery.Client(project=gcp_project_id).from_service_account_json(gcp_service_account_key)

# Setting the table we want from the source dataset
select_table = 'cleaned_full_polls_combined_national_results_2004_2019'

# SQL query for querying Big Query and fetching entire table
query = f"""
    SELECT *
    FROM `{gcp_project_id}.{bq_cleaned_dataset}.{select_table}`
"""

In [5]:
# Use BQ client to create DF from the selected table
data = bq_client.query(query).to_dataframe()

In [6]:
# Replace any found NaN values with 0
data.replace(np.nan, 0, inplace=True)

In [7]:
# Add a field 'poll length' that shows number of days the poll was held for
data['poll_length'] = pd.to_datetime(data.enddate) - pd.to_datetime(data.startdate)
data['poll_length'] = data['poll_length'].dt.days

In [8]:
# Divide forecasts by 100 to create values between 0-1
for column in ['BRX_FC', 'CON_FC', 'GRE_FC', 'LAB_FC', 'LIB_FC', 'NAT_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC']:
    data[column] = data[column] / 100

In [9]:
# Divide actuals by 100 to create values between 0-1
for column in ['BRX_ACT', 'CON_ACT', 'GRE_ACT', 'LIB_ACT', 'LAB_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT', 'OTH_PERCENTAGE']:
    data[column] = data[column] / 100

In [10]:
# Drop columns we don't need
data = data.drop(columns=['startdate', 'enddate', 'pollster'])

In [11]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'days_to_elec', 'poll_length']
num_transformer = MinMaxScaler()

In [12]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = OrdinalEncoder(categories = [['F','D-','D','D+','C-','B','B+','A-']])

In [13]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [14]:
# Fit transform preprocessing pipeline to data
data_processed = preproc_pipeline.fit_transform(data)

In [15]:
# Check feature names make sense
data_processed = pd.DataFrame(
    data_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [16]:
# Build our feature matrix, by dropping irrelevant and y columns
X = data_processed.drop(columns=['next_elec_date','NAT_ACT', 'BRX_ACT', 'CON_ACT',
       'GRE_ACT', 'LIB_ACT', 'LAB_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT',
       'OTH_PERCENTAGE'])

In [ ]:
# Build our target matrix, retaining each party share vote columns only
y = data_processed[['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT',
                    'NAT_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_PERCENTAGE']]

In [ ]:
# Handle data splitting
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# Handle target extraction for test and train sets respectively
y_train_LAB = y_train['LAB_ACT']
y_train_CON = y_train['CON_ACT']
y_train_LIB = y_train['LIB_ACT']
y_train_GRE = y_train['GRE_ACT']
y_train_BRX = y_train['BRX_ACT']
y_train_NAT = y_train['NAT_ACT']
y_train_SNP = y_train['SNP_ACT']
y_train_UKI = y_train['UKI_ACT']
y_train_PLC = y_train['PLC_ACT']
y_train_OTH = y_train['OTH_PERCENTAGE']

y_test_LAB = y_test['LAB_ACT']
y_test_CON = y_test['CON_ACT']
y_test_LIB = y_test['LIB_ACT']
y_test_GRE = y_test['GRE_ACT']
y_test_BRX = y_test['BRX_ACT']
y_test_NAT = y_test['NAT_ACT']
y_test_SNP = y_test['SNP_ACT']
y_test_UKI = y_test['UKI_ACT']
y_test_PLC = y_test['PLC_ACT']
y_test_OTH = y_test['OTH_PERCENTAGE']

In [ ]:
# Instantiate XGBRegressor Model
xgb_regression_model = XGBRegressor()

In [ ]:
# Instantiate the model for each party
model_LAB = xgb_regression_model
model_CON = xgb_regression_model
model_LIB = xgb_regression_model
model_GRE = xgb_regression_model
model_BRX = xgb_regression_model
model_NAT = xgb_regression_model
model_SNP = xgb_regression_model
model_UKI = xgb_regression_model
model_PLC = xgb_regression_model
model_OTH = xgb_regression_model

In [ ]:
# Handle model training
model_LAB.fit(X_train, y_train_LAB)
model_CON.fit(X_train, y_train_CON)
model_LIB.fit(X_train, y_train_LIB)
model_GRE.fit(X_train, y_train_GRE)
model_BRX.fit(X_train, y_train_BRX)
model_NAT.fit(X_train, y_train_NAT)
model_SNP.fit(X_train, y_train_SNP)
model_UKI.fit(X_train, y_train_UKI)
model_PLC.fit(X_train, y_train_PLC)
model_OTH.fit(X_train, y_train_OTH)

In [18]:
xgboost = XGBRegressor(learning_rate=0.3, n_estimators=1000,
                                     max_depth=3, subsample=0.7,
                                     objective='reg:linear', nthread=-1,
                                     )

# Score of the model
LAB_score = cross_val_score(xgboost,np.array(X),y_LAB, cv=5).mean()

/Users/oscar/.pyenv/versions/3.10.6/envs/election_predictor/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:58:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/oscar/.pyenv/versions/3.10.6/envs/election_predictor/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:58:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/oscar/.pyenv/versions/3.10.6/envs/election_predictor/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:58:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/Users/oscar/.pyenv/versions/3.10.6/envs/election_predictor/lib/python3.1